In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os 
import cv2 
from PIL import Image
from tqdm.auto import tqdm 
import glob as gb 

In [17]:
# !pip3 install opencv-python
# pip install tqdm

In [16]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.optim import Adam , lr_scheduler 
import torchvision 
from torchvision import datasets 
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader , Dataset
from torchsummary import summary
from torchmetrics import ConfusionMatrix 
from mlxtend.plotting import plot_confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score

In [18]:
# pip install torchvision
# !pip3 install torchsummary
# !pip3 install torchmetrics
# pip install mlxtend

In [22]:
#pip install numpy==1.21.1

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\ayush\Alzheimer\env\Lib\site-packages\pip\_internal\cli\base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "C:\Users\ayush\Alzheimer\env\Lib\site-packages\pip\_internal\cli\req_command.py", line 245, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ayush\Alzheimer\env\Lib\site-packages\pip\_internal\commands\install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "C:\Users\ayush\Alzheimer\env\Lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "C:\Users\ayush\Alzheimer\env\Lib\site-packages\pip\_vendor\resolvelib\resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_r


     ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
     --------------------------------------- 0.0/10.3 MB 330.3 kB/s eta 0:00:31
     - -------------------------------------- 0.4/10.3 MB 4.5 MB/s eta 0:00:03
     ----- ---------------------------------- 1.3/10.3 MB 10.3 MB/s eta 0:00:01
     ------- -------------------------------- 1.8/10.3 MB 10.6 MB/s eta 0:00:01
     -------- ------------------------------- 2.1/10.3 MB 10.3 MB/s eta 0:00:01
     -------- ------------------------------- 2.1/10.3 MB 10.3 MB/s eta 0:00:01
     -------- ------------------------------- 2.1/10.3 MB 10.3 MB/s eta 0:00:01
     -------- ------------------------------- 2.1/10.3 MB 10.3 MB/s eta 0:00:01
     -------- ------------------------------- 2.1/10.3 MB 5.2 MB/s eta 0:00:02
     --------- ------------------------------ 2.5/10.3 MB 5.5 MB/s eta 0:00:02
     ------------ --------------------------- 3.1/10.3 MB 6.2 MB/s eta 0:00:02
     --------------- ------------------------ 3.9/1

In [25]:
# def load_dataset(data_dir):
#     images = []
#     labels = []
#     label_names = os.listdir(data_dir)
    
#     for label in label_names:
#         label_dir = os.path.join(data_dir, label)
#         if os.path.isdir(label_dir):
#             for img_name in os.listdir(label_dir):
#                 img_path = os.path.join(label_dir, img_name)
#                 # Read the image (use cv2.imread, PIL.Image.open, etc.)
#                 image = cv2.imread(img_path)
#                 if image is not None:
#                     images.append(image)
#                     labels.append(label)
    
#     #return np.array(images), np.array(labels)
#     return images, labels

# # Specify the directory where your dataset is located
# data_dir = 'AugmentedAlzheimerDataset'

# # Load the dataset
# images, labels = load_dataset(data_dir)

# print(f"Loaded {len(images)} images with labels")

Loaded 33984 images with labels


In [38]:
# images = pd.DataFrame(images).to_numpy(dtype = float32) shows an error i am not able to rectify

In [39]:
def load_dataset(data_dir, image_size=(128, 128)):
    images = []
    labels = []
    label_names = os.listdir(data_dir)
    
    for label in label_names:
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                # Read the image
                image = cv2.imread(img_path)
                if image is not None:
                    # Resize the image to the fixed size
                    image = cv2.resize(image, image_size)
                    images.append(image)
                    labels.append(label)
    
    # Convert lists to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

# Specify the directory where your dataset is located
data_dir = 'AugmentedAlzheimerDataset'

# Load the dataset
images, labels = load_dataset(data_dir)

print(f"Loaded {len(images)} images with labels")
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Loaded 33984 images with labels
Images shape: (33984, 128, 128, 3)
Labels shape: (33984,)


In [40]:
images[0]

array([[[7, 7, 7],
        [7, 7, 7],
        [7, 7, 7],
        ...,
        [7, 7, 7],
        [7, 7, 7],
        [7, 7, 7]],

       [[7, 7, 7],
        [7, 7, 7],
        [7, 7, 7],
        ...,
        [7, 7, 7],
        [7, 7, 7],
        [7, 7, 7]],

       [[7, 7, 7],
        [7, 7, 7],
        [7, 7, 7],
        ...,
        [7, 7, 7],
        [7, 7, 7],
        [7, 7, 7]],

       ...,

       [[7, 7, 7],
        [7, 7, 7],
        [7, 7, 7],
        ...,
        [7, 7, 7],
        [7, 7, 7],
        [7, 7, 7]],

       [[7, 7, 7],
        [7, 7, 7],
        [7, 7, 7],
        ...,
        [7, 7, 7],
        [7, 7, 7],
        [7, 7, 7]],

       [[7, 7, 7],
        [7, 7, 7],
        [7, 7, 7],
        ...,
        [7, 7, 7],
        [7, 7, 7],
        [7, 7, 7]]], dtype=uint8)

In [41]:
labels[0]

'MildDemented'

In [42]:
class_labels = np.unique(labels)

In [43]:
class_labels

array(['MildDemented', 'ModerateDemented', 'NonDemented',
       'VeryMildDemented'], dtype='<U16')